# VQE with UCC ansatz

In [1]:
import openfermion as of

In [2]:
import numpy as np

In [3]:
from pytket import Circuit

## Hardware-Efficient Ansatz

For H2 model system with bond length 0.75A, sto3g basis, Jordan-Wigner encoding, with no qubit reduction or orbital freezing - taken from Qiskit Aqua/Pennylane

In [4]:
hamiltonian = (
    -0.8153001706270075 * of.QubitOperator("")
    + 0.16988452027940318 * of.QubitOperator("Z0")
    + -0.21886306781219608 * of.QubitOperator("Z1")
    + 0.16988452027940323 * of.QubitOperator("Z2")
    + -0.2188630678121961 * of.QubitOperator("Z3")
    + 0.12005143072546047 * of.QubitOperator("Z0 Z1")
    + 0.16821198673715723 * of.QubitOperator("Z0 Z2")
    + 0.16549431486978672 * of.QubitOperator("Z0 Z3")
    + 0.16549431486978672 * of.QubitOperator("Z1 Z2")
    + 0.1739537877649417 * of.QubitOperator("Z1 Z3")
    + 0.12005143072546047 * of.QubitOperator("Z2 Z3")
    + 0.04544288414432624 * of.QubitOperator("X0 X1 X2 X3")
    + 0.04544288414432624 * of.QubitOperator("X0 X1 Y2 Y3")
    + 0.04544288414432624 * of.QubitOperator("Y0 Y1 X2 X3")
    + 0.04544288414432624 * of.QubitOperator("Y0 Y1 Y2 Y3") # Pauli strings of local qubit operators
)
nuclear_repulsion_energy = 0.70556961456

In [5]:
def hardware_eff_ans(params):
    ansatz = Circuit(4)  # build 4 qubit circuit
    for i in range(4):
        ansatz.Ry(params[i], i)   # Ry gate with (coeff, qubit index)
    for i in range(3):
        ansatz.CX(i, i + 1)       # CX gate with (control, target)
    for i in range(4):
        ansatz.Ry(params[4 + i], i)
    return ansatz

In [6]:
from pytket.circuit.display import render_circuit_jupyter as draw

In [7]:
# single parameter value

arg_values = [
    -7.31158201e-02,
    -1.64514836e-04,
    1.12585591e-03,
    -2.58367544e-03,
    1.00006068e00,
    -1.19551357e-03,
    9.99963988e-01,
    2.53283285e-03,
]

In [8]:
circuit = hardware_eff_ans(arg_values)
draw(circuit) # ansatz circuit at t=0

In [9]:
from pytket.extensions.qiskit import AerBackend # Qiskit Aer package qasm_simulator
from pytket.utils.expectations import expectation_from_counts

In [10]:
backend = AerBackend()

In [11]:
def objective(params):
    energy = 0
    for term, coeff in hamiltonian.terms.items():  # iterate through each Pauli term and its coefficient
        if not term:
            energy += coeff # add on constant nuclear repulsion term
            continue

        circ = hardware_eff_ans(params) # create circuit with ansatz
        circ.add_c_register("c", len(term)) # add classical registers for measurements - 1 per term

        for i, (q, pauli) in enumerate(term):
            if pauli == "X": 
                circ.H(q)    # Hadamard for X measurement
            elif pauli == "Y": 
                circ.V(q)    # V gate for Y measurement
            circ.Measure(q, i)
        compiled_circ = backend.get_compiled_circuit(circ)
        counts = backend.run_circuit(compiled_circ, n_shots=4000).get_counts()
        energy += coeff * expectation_from_counts(counts)
    return energy + nuclear_repulsion_energy

In [12]:
energy = objective(arg_values)
print(energy)

-1.1368263656149007


## UCC ansatz

In [13]:
from pytket.circuit import Qubit
from pytket.pauli import Pauli, QubitPauliString

In [14]:
q = [Qubit(i) for i in range(4)]
xyii = QubitPauliString([q[0], q[1]], [Pauli.X, Pauli.Y])
yxii = QubitPauliString([q[0], q[1]], [Pauli.Y, Pauli.X])
iixy = QubitPauliString([q[2], q[3]], [Pauli.X, Pauli.Y])
iiyx = QubitPauliString([q[2], q[3]], [Pauli.Y, Pauli.X])
xxxy = QubitPauliString(q, [Pauli.X, Pauli.X, Pauli.X, Pauli.Y])
xxyx = QubitPauliString(q, [Pauli.X, Pauli.X, Pauli.Y, Pauli.X])
xyxx = QubitPauliString(q, [Pauli.X, Pauli.Y, Pauli.X, Pauli.X])
yxxx = QubitPauliString(q, [Pauli.Y, Pauli.X, Pauli.X, Pauli.X])
yyyx = QubitPauliString(q, [Pauli.Y, Pauli.Y, Pauli.Y, Pauli.X])
yyxy = QubitPauliString(q, [Pauli.Y, Pauli.Y, Pauli.X, Pauli.Y])
yxyy = QubitPauliString(q, [Pauli.Y, Pauli.X, Pauli.Y, Pauli.Y])
xyyy = QubitPauliString(q, [Pauli.X, Pauli.Y, Pauli.Y, Pauli.Y])

In [15]:
singles_a = {xyii: 1.0, yxii: -1.0}
singles_b = {iixy: 1.0, iiyx: -1.0}
doubles = {
    xxxy: 0.25,
    xxyx: -0.25,
    xyxx: 0.25,
    yxxx: -0.25,
    yyyx: -0.25,
    yyxy: 0.25,
    yxyy: -0.25,
    xyyy: 0.25,
}

In [16]:
def add_operator_term(circuit: Circuit, term: QubitPauliString, angle: float):
    qubits = []
    for q, p in term.map.items():
        if p != Pauli.I:
            qubits.append(q)
            if p == Pauli.X:
                circuit.H(q)
            elif p == Pauli.Y:
                circuit.V(q)
    for i in range(len(qubits) - 1):
        circuit.CX(i, i + 1)
    circuit.Rz(angle, len(qubits) - 1)
    for i in reversed(range(len(qubits) - 1)):
        circuit.CX(i, i + 1)
    for q, p in term.map.items():
        if p == Pauli.X:
            circuit.H(q)
        elif p == Pauli.Y:
            circuit.Vdg(q)

In [17]:
def ucc(params):
    ansatz = Circuit(4)
    # Set initial reference state
    ansatz.X(1).X(3)
    # Evolve by excitations
    for term, coeff in singles_a.items():
        add_operator_term(ansatz, term, coeff * params[0])
    for term, coeff in singles_b.items():
        add_operator_term(ansatz, term, coeff * params[1])
    for term, coeff in doubles.items():
        add_operator_term(ansatz, term, coeff * params[2])
    return ansatz

In [18]:
from pytket.circuit import PauliExpBox
from pytket.passes import DecomposeBoxes

In [19]:
def add_excitation(circ, term_dict, param):
    for term, coeff in term_dict.items():
        qubits, paulis = zip(*term.map.items())
        pbox = PauliExpBox(paulis, coeff * param)
        circ.add_gate(pbox, qubits)

In [20]:
# boxed gates
def ucc(params):
    ansatz = Circuit(4)
    ansatz.X(1).X(3)
    add_excitation(ansatz, singles_a, params[0])
    add_excitation(ansatz, singles_b, params[1])
    add_excitation(ansatz, doubles, params[2])
    DecomposeBoxes().apply(ansatz)
    return ansatz

In [21]:
circuit1 = ucc(arg_values)
draw(circuit1) # ansatz circuit at t=0

In [22]:
from pytket.utils.operators import QubitPauliOperator

In [23]:
pauli_sym = {"I": Pauli.I, "X": Pauli.X, "Y": Pauli.Y, "Z": Pauli.Z}

In [24]:
def qps_from_openfermion(paulis):
    """Convert OpenFermion tensor of Paulis to pytket QubitPauliString."""
    qlist = []
    plist = []
    for q, p in paulis:
        qlist.append(Qubit(q))
        plist.append(pauli_sym[p])
    return QubitPauliString(qlist, plist)

In [25]:
def qpo_from_openfermion(openf_op):
    """Convert OpenFermion QubitOperator to pytket QubitPauliOperator."""
    tk_op = dict()
    for term, coeff in openf_op.terms.items():
        string = qps_from_openfermion(term)
        tk_op[string] = coeff
    return QubitPauliOperator(tk_op)

In [26]:
hamiltonian_op = qpo_from_openfermion(hamiltonian)

In [27]:
from pytket.utils.expectations import get_operator_expectation_value

In [28]:
def objective(params):
    circ = ucc(params)
    return (
        get_operator_expectation_value(circ, hamiltonian_op, backend, n_shots=4000)
        + nuclear_repulsion_energy
    )

In [29]:
arg_values = [-3.79002933e-05, 2.42964799e-05, 4.63447157e-01]

In [30]:
energy = objective(arg_values)
print(energy)

(-1.1398580992457492+0j)


Decomposing into basic gates is not particularly effective as scales exponentially with number of Pauli strings - use Pytket circuit simplification. The example here uses decomposing PauliExpBox into basic gates and then applying FullPeepholeOptimize to remove redundancies. We can examine the complexity by looking at the number of two-qubit gates before and after simplification. 

In [31]:
from pytket import OpType
from pytket.passes import FullPeepholeOptimise

In [32]:
print("CX count before", circuit1.n_gates_of_type(OpType.CX))
print("CX depth before", circuit1.depth_by_type(OpType.CX))

CX count before 56
CX depth before 36


In [33]:
FullPeepholeOptimise().apply(circuit1)
print("CX count after FPO", circuit1.n_gates_of_type(OpType.CX))
print("CX depth after FPO", circuit1.depth_by_type(OpType.CX))
draw(circuit1)

CX count after FPO 26
CX depth after FPO 23


UCC ansaetze are made out of exponentiated tensors of Pauli matrices, and as such can be grouped in terms of commuting terms.

In [34]:
from pytket.passes import PauliSimp

In [35]:
test_circuit = ucc(arg_values)
PauliSimp().apply(test_circuit)

True

In [36]:
print("CX count after PS", test_circuit.n_gates_of_type(OpType.CX))
print("CX depth after PS", test_circuit.depth_by_type(OpType.CX))

CX count after PS 26
CX depth after PS 19


In [37]:
FullPeepholeOptimise().apply(test_circuit)
print("CX count after PS+FPO", test_circuit.n_gates_of_type(OpType.CX))
print("CX depth after PS+FPO", test_circuit.depth_by_type(OpType.CX))
draw(test_circuit)

CX count after PS+FPO 15
CX depth after PS+FPO 12


In [38]:
def objective(params):
    circ = ucc(params)
    PauliSimp().apply(circ)
    FullPeepholeOptimise().apply(circ)
    return (
        get_operator_expectation_value(circ, hamiltonian_op, backend, n_shots=4000)
        + nuclear_repulsion_energy
    )

In [39]:
from pytket.passes import GuidedPauliSimp
from pytket.utils import gen_term_sequence_circuit   # group terms together in collection of PauliExpBox 

In [40]:
def ucc(params):
    singles_params = {qps: params[0] * coeff for qps, coeff in singles.items()}
    doubles_params = {qps: params[1] * coeff for qps, coeff in doubles.items()}
    excitation_op = QubitPauliOperator({**singles_params, **doubles_params})
    reference_circ = Circuit(4).X(1).X(3)
    ansatz = gen_term_sequence_circuit(excitation_op, reference_circ)
    GuidedPauliSimp().apply(ansatz)
    FullPeepholeOptimise().apply(ansatz)
    return ansatz

In [41]:
from sympy import symbols

In [42]:
syms = symbols("p0 p1 p2")
singles_a_syms = {qps: syms[0] * coeff for qps, coeff in singles_a.items()}
singles_b_syms = {qps: syms[1] * coeff for qps, coeff in singles_b.items()}
doubles_syms = {qps: syms[2] * coeff for qps, coeff in doubles.items()}
excitation_op = QubitPauliOperator({**singles_a_syms, **singles_b_syms, **doubles_syms})
ucc_ref = Circuit(4).X(1).X(3)
ucc = gen_term_sequence_circuit(excitation_op, ucc_ref)
GuidedPauliSimp().apply(ucc)
FullPeepholeOptimise().apply(ucc)

True

In [43]:
draw(ucc)

In [44]:
def objective(params):
    circ = ucc.copy()
    sym_map = dict(zip(syms, params))
    circ.symbol_substitution(sym_map)
    return (
        get_operator_expectation_value(circ, hamiltonian_op, backend, n_shots=4000)
        + nuclear_repulsion_energy
    )

In [45]:
from pytket.partition import PauliPartitionStrat  # minimize number of gates with combining measurements for terms that mutually commute

In [46]:
def objective(params):
    circ = ucc.copy()
    sym_map = dict(zip(syms, params))
    circ.symbol_substitution(sym_map)
    return (
        get_operator_expectation_value(
            circ,
            hamiltonian_op,
            backend,
            n_shots=4000,
            partition_strat=PauliPartitionStrat.CommutingSets,
        )
        + nuclear_repulsion_energy
    ).real

In [47]:
objective(arg_values)

-1.1315286587205935

In [ ]:
from scipy.optimize import minimize
initial_params = [1e-4, 1e-4, 4e-1]
result = minimize(objective, initial_params, method="Nelder-Mead")
print("Final parameter values", result.x)
print("Final energy value", result.fun)